In [1]:
#import packages
import pandas as pd
from datetime import datetime

In [2]:
#Assign Global Mobility Report to url
url = "https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv"

In [3]:
cols = [
        "country_region",
        "sub_region_1",
        "sub_region_2",
        "metro_area",
        "iso_3166_2_code",
        "census_fips_code",
        "date",
        "retail_and_recreation_percent_change_from_baseline",
        "grocery_and_pharmacy_percent_change_from_baseline",
        "parks_percent_change_from_baseline",
        "transit_stations_percent_change_from_baseline",
        "workplaces_percent_change_from_baseline",
        "residential_percent_change_from_baseline"
]

In [8]:
df = pd.read_csv(url, usecols=cols, low_memory=False)

In [9]:
#Convert date column to days of the year
df['date'] = pd.to_datetime(df['date'], format="%Y/%m/%d", utc=True)
df['date'] = df['date'].dt.dayofyear

In [14]:
#Standardise country names to OWID country names
# country_mapping = pd.read_csv("mobility_to_owid_countries.csv")
country_mapping = pd.read_csv("../input/owid/eu_countries.csv")
df = country_mapping.merge(df, on="country_region")
country_mapping.head()

,Country,Region
0,Austria,European Union
1,Belgium,European Union
2,Bulgaria,European Union
3,Croatia,European Union
4,Cyprus,European Union


In [120]:
#Remove subnational data, keeping only country figures
filter_cols = ["sub_region_1", "sub_region_2","metro_area","iso_3166_2_code","census_fips_code"]
country_mobility = df[df[filter_cols].isna().all(1)]

In [121]:
#Delete columns
country_mobility = country_mobility.drop(columns=["country_region", "sub_region_1", "sub_region_2","metro_area","census_fips_code","iso_3166_2_code"])

In [122]:
#Assign new column names
rename_dict = {
        "date": "Year",
        "retail_and_recreation_percent_change_from_baseline": "Retail & Recreation",
        "grocery_and_pharmacy_percent_change_from_baseline": "Grocery & Pharmacy",
        "parks_percent_change_from_baseline": "Parks",
        "transit_stations_percent_change_from_baseline": "Transit Stations",
        "workplaces_percent_change_from_baseline": "Workplaces",
        "residential_percent_change_from_baseline": "Residential"
    }

In [123]:
#Rename columns
country_mobility = country_mobility.rename(columns=rename_dict)

In [124]:
country_mobility

,Country,Year,Retail & Recreation,Grocery & Pharmacy,Parks,Transit Stations,Workplaces,Residential
0,United Arab Emirates,46,0.0,4.0,5.0,0.0,2.0,1.0
1,United Arab Emirates,47,1.0,4.0,4.0,1.0,2.0,1.0
2,United Arab Emirates,48,-1.0,1.0,5.0,1.0,2.0,1.0
3,United Arab Emirates,49,-2.0,1.0,5.0,0.0,2.0,1.0
4,United Arab Emirates,50,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...
1321317,Zimbabwe,203,-11.0,-4.0,-3.0,-30.0,-14.0,23.0
1321318,Zimbabwe,204,-22.0,-14.0,-13.0,-40.0,-19.0,28.0
1321319,Zimbabwe,205,-22.0,-12.0,-12.0,-36.0,-19.0,28.0
1321320,Zimbabwe,206,-30.0,-18.0,-23.0,-47.0,-19.0,30.0


In [125]:
#Save to files
country_mobility.to_csv('Google Mobility Trends (2020).csv', index=False)